# Laboratorio 6 Parte 2

### Reducción de dimensión: PCA y LDA

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Estudiantes

#### Primer Integrante: Jorge Andres Hiler Ricardo
#### Segundo Integrante: Santiago Gaviria Zapata


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.


Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: 
`!pip install mlxtend`
También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/





In [1]:
pip install qgrid

You should consider upgrading via the '/home/sady/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from mlxtend.feature_extraction import LinearDiscriminantAnalysis as LDA

import time


Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography



Analice la base de datos, sus características, su variable de salida y el contexto del problema.



Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [3]:
#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


## Ejercicio 1: Entrenamiento sin extracción de características

En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [4]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    print('train_index', train_index)
    print('test_index', test_index)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nEficiencia en validación sin extracción: " + str((1-np.mean(Errores))*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 

train_index [ 213  214  215 ... 2123 2124 2125]
test_index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212]
train_index [   0    1    2 ... 2123 2124 2125]
test_index [213 214 215 216 

train_index [   0    1    2 ... 2123 2124 2125]
test_index [1702 1703 1704 1705 1706 1707 1708 1709 1710 1711 1712 1713 1714 1715
 1716 1717 1718 1719 1720 1721 1722 1723 1724 1725 1726 1727 1728 1729
 1730 1731 1732 1733 1734 1735 1736 1737 1738 1739 1740 1741 1742 1743
 1744 1745 1746 1747 1748 1749 1750 1751 1752 1753 1754 1755 1756 1757
 1758 1759 1760 1761 1762 1763 1764 1765 1766 1767 1768 1769 1770 1771
 1772 1773 1774 1775 1776 1777 1778 1779 1780 1781 1782 1783 1784 1785
 1786 1787 1788 1789 1790 1791 1792 1793 1794 1795 1796 1797 1798 1799
 1800 1801 1802 1803 1804 1805 1806 1807 1808 1809 1810 1811 1812 1813
 1814 1815 1816 1817 1818 1819 1820 1821 1822 1823 1824 1825 1826 1827
 1828 1829 1830 1831 1832 1833 1834 1835 1836 1837 1838 1839 1840 1841
 1842 1843 1844 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855
 1856 1857 1858 1859 1860 1861 1862 1863 1864 1865 1866 1867 1868 1869
 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883
 1884 1885 1886 18



1.1 Cuando se aplica PCA ¿es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/:Si, debido a que es un técnica que usa medidas de distancia y cuando se están calculando las componentes uno de los criterios que deben cumplirse es que la distancia entre la proyección y la componente sea minima. También debido a que esta técnica se enfoca en las características de mayor variación.
    
1.2 La proyección de los datos que realiza PCA busca optimizar un medida, ¿Cuál? Explique.

R/:Busca minimizar la distancia entre cada punto y su proyección 

## Ejercicio 2: Entrenamiento con extracción de características

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características. https://rasbt.github.io/mlxtend/user_guide/feature_extraction/

In [0]:
'''
Feature Extraction Function
#Recibe 2 parámetros: 
1. el tipo de método de extracción (pca o lda como string),
2. el número componentes (para pca) o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.
'''

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

In [68]:

def SVM_PCA_LDA(tipo, n, X): 

  #Para calcular el costo computacional
  tiempo_i = time.time()

  #Estandarizamos los datos
  X = standardize(X)

  #Implemetamos la metodología de validación cross validation con 10 folds

  Errores = np.ones(10)
  j = 0
  kf = KFold(n_splits=10)

  for train_index, test_index in kf.split(X):
      
      #Aquí se aplica la extracción de características por PCA
      #Complete el código
      
      #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1
      ex =  extract_features(tipo,n)

      #Fit de PCA
      #Complete el código con el fit correspondiente
      ex = ex.fit(X)
      
      #Transforme las variables y genere el nuevo espacio de características de menor dimensión
      X_ex = ex.transform(X)#complete el código aquí para hacer la transformación
      
      
      #Aquí se aplica la extracción de características por LDA
      
      #OPCIONAL
      '''
      #ex = #Complete el código llamando el método extract_features.Tenga en cuenta lo que le pide el ejercicio 3.1

      #Fit de LDA
      ex = #Complete el código con el fit correspondiente
      
      #Transforme las variables y genere el nuevo espacio de características de menor dimensión
      X_ex = #complete el código aquí para hacer la transformación
      '''
      
      #Se aplica CV-10
      
      X_train, X_test = X_ex[train_index], X_ex[test_index]
      y_train, y_test = Y[train_index], Y[test_index]  
    
      #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA
      
      ######
      
      # Entrenamiento el modelo.
      model = clf.fit(X_train,y_train)

      # Validación del modelo
      ypred = model.predict(X_test)
      
      #######

      Errores[j] = classification_error(ypred, y_test)
      j+=1
          

  print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

  print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

  print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

  return np.mean(Errores), np.std(Errores), (1-np.mean(Errores))*100, time.time()-tiempo_i

print(SVM_PCA_LDA('pca', 2, X))



Error de validación aplicando extracción: 0.22142793870139074 +/- 0.17001475908401714

Eficiencia en validación aplicando extracción: 77.85720612986093%


Tiempo total de ejecución: 0.7970623970031738 segundos.
(0.22142793870139074, 0.17001475908401714, 77.85720612986093, 0.7971689701080322)


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

In [66]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + PCA','SVM + PCA','SVM + PCA','SVM + PCA']),
    '# de características seleccionadas' : pd.Series(['N/A',2,10,19,21]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Eficiencia"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)

#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

for i in df_types.index:
  if (str(i[1]) == 'N/A'):
    ERR, IC, EF, TM = '0.0771281','0.04353333','92.66365','0.59319067'
  else:                
    ERR, IC, EF, TM = SVM_PCA_LDA('pca', i[1], X)
    
  df_types["Error de validación"][i] = ERR
  df_types["IC(std)"][i] = IC
  df_types["Eficiencia"][i] = EF
  df_types["Tiempo de ejecución"][i] = TM

df_types



Error de validación aplicando extracción: 0.22142793870139074 +/- 0.17001475908401714

Eficiencia en validación aplicando extracción: 77.85720612986093%


Tiempo total de ejecución: 0.8120348453521729 segundos.

Error de validación aplicando extracción: 0.09169324120825582 +/- 0.061564467997647386

Eficiencia en validación aplicando extracción: 90.83067587917442%


Tiempo total de ejecución: 0.68729567527771 segundos.

Error de validación aplicando extracción: 0.07193949862698203 +/- 0.04841386304281984

Eficiencia en validación aplicando extracción: 92.8060501373018%


Tiempo total de ejecución: 0.7032825946807861 segundos.

Error de validación aplicando extracción: 0.07430463282841704 +/- 0.04391645244688941

Eficiencia en validación aplicando extracción: 92.5695367171583%


Tiempo total de ejecución: 0.7013306617736816 segundos.


Error de validación  ... Tiempo de ejecución
Tecnica            # de características seleccionadas                      ...                    
SVM sin extracción N/A                                          0.0771281  ...          0.59319067
SVM + PCA          2                                             0.221428  ...             0.81215
                   10                                           0.0916932  ...            0.687452
                   19                                           0.0719395  ...            0.703414
                   21                                           0.0743046  ...            0.701445

[5 rows x 4 columns]

3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: No es viable. La mejora que se obtiene, cuando se acerca al numero de caracteristicas (23), es insignifcante y aumenta el costo computacional ya que toma más tiempo.



3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: LDA tiene como criterio reducir la dimensionalidad maximizando la separación entre los grupos de datos, mientras que en PCA el criterio es reducir la dimensión enfocandose en las características de que representan mayor variación 

3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/: El método de selección implica desechar unas características y considerar sola los mejores predictores a diferencia de la extracción que busca tranformar el conjunto de caracteristicas en un conjunto reducido.

